Voici mon application :Analyseur de Cv.
Le format du cv doit être .pdf 
Voici le chemin d'acces que vous devez changer pour analyser le pdf de votre choix

In [541]:
cheminCV="cv-finance.pdf"

Import des librairies et du modele d'entrainement puis de la fonction d'extraction du texte


In [542]:
import spacy
import pdfplumber
from spacy.training.example import Example
import json
import re


def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text(encoding='utf-8')  # Spécifie l'encodage
    return text







Fonction pour detecter le domaine d'activité

In [543]:
def detect_domain(text):
    domains = {
        'Informatique': ['informatique',  'programmation', 'développement WEB', 'développement mobile',
                        'intelligence artificielle', 'IA', 'machine learning', 'réseau', 'cybersécurité',
                        'sécurité informatique', 'cyberdéfense', 'cyberattaque', 'cyberattaque',
                        'cybercriminalité', 'cybercriminalité', 'cyberespionnage', 'cyberespionnage',
                        'cyberterrorisme','programmation web', 'programmation mobile', 'programmation web','language de programmation',
                        'programmation orientée objet', 'programmation procédurale', 'programmation fonctionnelle', 
                        'programmation impérative', 'programmation déclarative', 'programmation concurrente', 'programmation parallèle',
                        'programmation logique', 'programmation générique', 'programmation événementielle', 'programmation réactive', 
                        'programmation structurée', 'programmation modulaire', 'program','Langages de programmations', 'langage de programmation','php','JS','java','python','c++','c#','ruby','swift','kotlin','scala','perl','rust','typescript','dart','sql','html','css','xml','json','yaml','markdown'],
        'Commerce': ['commerce',  'marketing', 'vente', 'négociation', 'commerce international'],
        'Finance': ['finance', 'comptabilité', 'économie', 'gestion financière'],
        'Ressources Humaines': ['RH', 'ressources humaines', 'recrutement', 'formation'],
        'Ingénierie': ['ingénierie', 'ingénieur', 'technique', 'conception'],
        'Santé': ['santé', 'médical', 'soins', 'pharmacie'],
        'Éducation': ['éducation', 'enseignement', 'pédagogie', 'formation'],
        'Art et Culture': ['art', 'culture', 'musique', 'théâtre'],
        'Administration': ['administration', 'gestion administrative', 'bureautique'],
        'Environnement': ['environnement', 'écologie', 'développement durable'],
        'Communication': ['communication', 'relations publiques', 'médias', 'journalisme'],
        'Agriculture': ['agriculture', 'agronomie', 'agroalimentaire'],
        'Tourisme': ['tourisme', 'voyage', 'hôtellerie', 'guidage'],
        'Droit': ['droit', 'juridique', 'justice', 'avocat'],
        'Transport et Logistique': ['transport', 'logistique', 'supply chain', 'entreposage']
        # Ajoute d'autres domaines avec leurs mots-clés associés ici
    }

    text_lower = text.lower()
    domain_count = {domain: 0 for domain in domains}

    for domain, keywords in domains.items():
        for keyword in keywords:
            domain_count[domain] += text_lower.count(keyword)

    sorted_domains = sorted(domain_count.items(), key=lambda x: x[1], reverse=True)
    top_three_domains = sorted_domains[:3]

    return top_three_domains

Fonction pour detecter le numéro de téléphone

In [544]:

def extract_phone_numbers(text):
    phone_numbers = re.findall(r'\b\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}\b', text)
    return phone_numbers


Fonction pour detecter le type de contrat

In [545]:
def detect_contrat_type(text):
    contrat_types = {
        'Stage': ['stage', 'stage étudiant', 'stage de fin d\'études'],
        'Alternance': ['alternance'],
        'CDD': ['cdd', 'contrat à durée déterminée'],
        'CDI': ['cdi', 'contrat à durée indéterminée'],
        # Ajoute d'autres types de contrat avec leurs mots-clés associés ici
    }

    text_lower = text.lower()

    for contrat, keywords in contrat_types.items():
        for keyword in keywords:
            if keyword in text_lower:
                return contrat

    return None

Fonction pour detecter l'adresse mail 

In [546]:
def extract_emails(text):
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    return emails


Fonction pour detecter les langues parlés

In [547]:
def extract_languages(text):
    languages = re.findall(r'\b(?:français|anglais|espagnol|allemand)\b(?:\s(?:A|B|C)\d)?', text, flags=re.IGNORECASE)
    return languages

Fonction pour detecter les dates (a finir pour avoir les experience plus date reliés)

In [548]:
def extract_date_ranges(text):
    date_ranges = []
    # Expressions régulières pour les séparateurs '-' et '_'
    regex_dash = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\s-\s(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    regex_underscore = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\s_\s(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    regex_underscore_witout_space = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}_(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    regex_dash_witout_space = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}-(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    date_rangeM =r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\s–\s(?:Maintenant|(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4})'

    # Recherche des plages de dates dans le texte
    matches_dash = re.findall(regex_dash, text)
    matches_underscore = re.findall(regex_underscore, text)
    matches_underscore_witout_space = re.findall(regex_underscore_witout_space, text)
    matches_dash_witout_space = re.findall(regex_dash_witout_space, text)
    matches_date_rangeM = re.findall(date_rangeM, text)
    # Ajout des résultats des deux expressions régulières dans un seul tableau
    date_ranges.extend(matches_underscore_witout_space)
    date_ranges.extend(matches_underscore)
    date_ranges.extend(matches_dash)
    date_ranges.extend(matches_dash_witout_space)
    date_ranges.extend(matches_date_rangeM)

    return date_ranges

Fonction pour détecter une adresse 

In [549]:

def detect_addresses(text):
    # Expression régulière pour détecter une adresse française avec le style donné
    address_pattern = r'\b\d{1,4}(?:[,\s]+)?(?:\w+[,\s]+){1,3}(?:rue|avenue|boulevard|chemin|impasse|allée|place|route|quai)\b[\s,]+\b\w+\b.*?(?=[\n.,]|$)'


    # Recherche des adresses dans le texte
    addresses = re.findall(address_pattern, text, flags=re.IGNORECASE)

    return addresses



def detect_postal_codes(text):
   # Expression régulière pour détecter les lignes contenant un code postal à cinq chiffres
    postal_code_pattern = r'(\b\d{5}\b.*?)(?=[\n.,]|$)'

    # Recherche des lignes avec code postal dans le texte
    lines_with_postal_codes = re.findall(postal_code_pattern, text, flags=re.IGNORECASE | re.DOTALL)

    return lines_with_postal_codes



Affichage du texte extrait

In [550]:
pdf_path = cheminCV  # Remplacez ceci par votre chemin
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

12 rue de la Réussite
Da mien
75012 Paris
LAFINANCE
01 02 03 04 05
06 01 02 03 04
Je s uis a la recherche d’un CDD de 5 moins monemail@mail.com
mi nimun www.monsiteweb.com
FORMATION PROFIL
JANV 2014 – FEB 2016 Avec 10 ans d'expérience en analyse financière et une expertise en modélisation financière,
ISTEC PARIS évaluation d'entreprises et gestion de risques, je suis un analyste financier compétent et
MASTER EN FINANCE ET GESTION D’ENTREPRISE rigoureux, capable de fournir des analyses précises et des recommandations éclairées pour
Stage d’analyste financier, Société Générale aider les entreprises à atteindre leurs objectifs financiers à long terme
Corporate & Investment Banking, Paris, France
(été 2015)
EXPERIENCES PROFESSIONNELLES
APR 2012 – JULY 2016
UNIVERSITE PARIS-DAUPHINE
DEPUIS JUIN 2018
LICENCE D’ECONOMIE ET GESTION
ANALYSTITE FINANCIER PRINCIPAL
Analyste financier stagiaire, Rothschild & Co, Paris, GOLMAN SACHS, LONDRES, RU
France (été 2016) • Effectue des analyses financières

Code pour entrainer l'analyseur de PDF

In [551]:

# # Lecture des données annotées pour l'entraînement du modèle
# with open('train_data.json', 'r', encoding='utf-8') as file:
#     training_data = json.load(file)

# # Entraînement du modèle SpaCy
# nlp = spacy.blank("fr")
# ner = nlp.add_pipe("ner", last=True)

# # Ajout des labels, y compris les nouveaux labels "EMAIL", "TELEPHONE" et "COMPETENCE"
# ner.add_label("PER")
# ner.add_label("EXP")
# ner.add_label("DATE")
# ner.add_label("EMAIL")
# ner.add_label("TELEPHONE")
# ner.add_label("COMPETENCE")

# # Création des exemples pour l'entraînement du modèle
# examples = []
# for data in training_data:
#     text = data["text"]
#     entities = []
#     for entity in data["entities"]:
#         start, end, label = entity["start"], entity["end"], entity["label"]
#         entities.append((start, end, label))
#     examples.append(Example.from_dict(nlp.make_doc(text), {"entities": entities}))

# # Entraînement du modèle
# nlp.initialize(lambda: examples)
# for i in range(10):  # Nombre d'itérations d'entraînement
#     losses = {}
#     nlp.update(examples, drop=0.5, losses=losses)
#     print(losses)

# # Sauvegarde du modèle entraîné
# nlp.to_disk("modele_cv")
# # Chargement du modèle SpaCy entraîné
# nlp_loaded = spacy.load("modele_cv")




In [552]:
# # Extraction des informations du nouveau CV
# pdf_path = "CV-14.pdf"
# cv_text = extract_text_from_pdf(pdf_path)
# doc = nlp_loaded(cv_text)

# # Affichage des entités extraites du CV
# for ent in doc.ents:
#     print(ent.text, ent.label_)

text extraction mail et telephone

In [553]:
phone_numbers = extract_phone_numbers(extracted_text)
emails = extract_emails(extracted_text)


print("Numéros de téléphone trouvés :")
print(phone_numbers)


print("\nAdresses e-mail trouvées :")
print(emails)



print("\nLangues parlées :")
print(extract_languages(extracted_text))


dates = extract_date_ranges(extracted_text)
print("Plages de dates trouvées :", dates)



top_domains = detect_domain(extracted_text)

if top_domains:
    print("classement des domaines :")
    for domain, count in top_domains:
        if(count>1):
         print(f"{domain}: {count} occurrence(s)")
else:
    print("Aucun domaine identifié.")





    # Utilisation de la fonction pour détecter le type de contrat dans le texte extrait
contrat_detected = detect_contrat_type(extracted_text)

# Affichage du type de contrat détecté
if contrat_detected:
    print(f"Type de contrat détecté : {contrat_detected}")
else:
    print("Type de contrat non identifié.")





# Utilisation de la fonction pour détecter les adresses avec code postal dans le texte extrait
addresses_detected = detect_addresses(extracted_text)

# Affichage des adresses détectées avec code postal
if addresses_detected:
    print("Adresses françaises avec code postal trouvées :")
    for address in addresses_detected:
        print(address)
else:
    print("Aucune adresse française identifiée.")
    # Utilisation de la fonction pour détecter les codes postaux dans le texte extrait
postal_codes_detected = detect_postal_codes(extracted_text)

# Affichage des codes postaux détectés
if postal_codes_detected:
    print("Codes postaux détectés + ville :")
    for postal_code in postal_codes_detected:
        print(postal_code)
else:
    print("Aucun code postal identifié.")


Numéros de téléphone trouvés :
['01 02 03 04 05', '06 01 02 03 04']

Adresses e-mail trouvées :
['monemail@mail.com']

Langues parlées :
[]
Plages de dates trouvées : []
classement des domaines :
Art et Culture: 3 occurrence(s)
Finance: 2 occurrence(s)
Type de contrat détecté : Stage
Adresses françaises avec code postal trouvées :
12 rue de la Réussite
Codes postaux détectés + ville :
75012 Paris
